## Data Preprocessing

In [24]:
# Artificial Neural Network

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]


In [25]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [26]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [27]:
X = X.values
y = y.values

In [28]:
#Categorical data encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder_geo = LabelEncoder()
X[:,1] = labelEncoder_geo.fit_transform(X[:,1])

labelEncoder_gen = LabelEncoder()
X[:,2] = labelEncoder_geo.fit_transform(X[:,2])

In [29]:
# Creating dummy variables for category : country
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

In [30]:
X = X[:,1:] # to avoid dummy variable trap : 1 can be determined by other 2

In [31]:
# splitting training set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size =0.2, random_state=0)



In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [33]:
X_train.shape

(8000, 11)

In [34]:
X[0]

array([0.0000000e+00, 0.0000000e+00, 6.1900000e+02, 0.0000000e+00,
       4.2000000e+01, 2.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

In [35]:
import keras

In [36]:
from keras.models import Sequential

In [37]:
from keras.layers import Dense

## Evaluating, Improving and Tuning ANN

In [45]:
##including k cross validation

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

In [51]:
def build_classifier():
    classifier = Sequential()
    #adding input layer and first hidden layer
    #general concept : no. of hidden layers =  avg. of no. of nodes in input layer and output layer = (11 + 1)/2
    classifier.add(Dense(units = 6,init = 'uniform', activation = 'relu',input_dim=11 ))
    classifier.add(Dropout(p=0.1)) # add dropout to first hidden layer - 1 neuron deactivates every iteration
    #second hidden layer
    classifier.add(Dense(units = 6,init = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p=0.1)) # add dropout to second hidden layer
    classifier.add(Dense(units = 1,init = 'uniform', activation = 'sigmoid' ))
    classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return classifier


In [47]:
classifier = KerasClassifier(build_fn=build_classifier,batch_size = 10, nb_epoch=100)

In [48]:
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=1)
accuracies

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="relu", input_dim=11, kernel_initializer="uniform")`
  """
C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="relu", kernel_initializer="uniform")`
  
C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':
C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/1
800/800 [==============================] - 1s 1ms/step
Epoch 1/1
800/800 [==============================] - 1s 677us/step
Epoch 1/1
800/800 [==============================] - 1s 759us/step
Epoch 1/1
800/800 [==============================] - 1s 720us/step
Epoch 1/1
800/800 [==============================] - 1s 715us/step
Epoch 1/1
800/800 [==============================] - 1s 765us/step
Epoch 1/1
800/800 [==============================] - 1s 798us/step
Epoch 1/1
800/800 [==============================] - 1s 1ms/step
Epoch 1/1
800/800 [==============================] - 1s 1ms/step
Epoch 1/1
800/800 [==============================] - 1s 852us/step


array([0.78625   , 0.79      , 0.8       , 0.7825    , 0.81625   ,
       0.81      , 0.7875    , 0.79375   , 0.79875   , 0.79499999])

In [49]:
accuracies.mean()

0.7959999969601632

In [50]:
accuracies.std() # variance, its not bad, high variance means overfitting

0.010105690620981692

## Tuning the ANN - find best parameters

In [53]:
from sklearn.model_selection import GridSearchCV


In [70]:
def build_classifier_grid(optimizer_arg):
    classifier = Sequential()
    #adding input layer and first hidden layer
    #general concept : no. of hidden layers =  avg. of no. of nodes in input layer and output layer = (11 + 1)/2
    classifier.add(Dense(units = 6,init = 'uniform', activation = 'relu',input_dim=11 ))
    #classifier.add(Dropout(p=0.1)) # add dropout to first hidden layer - 1 neuron deactivates every iteration
    #second hidden layer
    classifier.add(Dense(units = 6,init = 'uniform', activation = 'relu'))
    #classifier.add(Dropout(p=0.1)) # add dropout to second hidden layer
    classifier.add(Dense(units = 1,init = 'uniform', activation = 'sigmoid' ))
    classifier.compile(optimizer=optimizer_arg, loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

In [66]:
classifier = KerasClassifier(build_fn=build_classifier_grid)
parameters = {'batch_size':[25,32], 
              'nb_epoch':[100,500],
              'optimizer_arg':['adam','rmsprop']                 
             }
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=parameters,
                           scoring='accuracy',cv=10)


In [67]:
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="relu", input_dim=11, kernel_initializer="uniform")`
  """
C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="relu", kernel_initializer="uniform")`
  
C:\Users\Rishabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/1
7200/7200 [==============================] - 2s 313us/step - loss: 0.5613 - acc: 0.7974
Epoch 1/1
7200/7200 [==============================] - 2s 331us/step - loss: 0.5544 - acc: 0.7967
Epoch 1/1
7200/7200 [==============================] - 3s 382us/step - loss: 0.5553 - acc: 0.7956
Epoch 1/1
7200/7200 [==============================] - 2s 295us/step - loss: 0.5511 - acc: 0.7971
Epoch 1/1
7200/7200 [==============================] - 2s 297us/step - loss: 0.5514 - acc: 0.7933
Epoch 1/1
7200/7200 [==============================] - 2s 299us/step - loss: 0.5789 - acc: 0.7933
Epoch 1/1
7200/7200 [==============================] - 2s 331us/step - loss: 0.5673 - acc: 0.7953
Epoch 1/1
7200/7200 [==============================] - 2s 313us/step - loss: 0.5773 - acc: 0.7950
Epoch 1/1
7200/7200 [==============================] - 2s 313us/step - loss: 0.5743 - acc: 0.7953
Epoch 1/1
7200/7200 [==============================] - 3s 368us/step - loss: 0.5649 - acc: 0.7961
Epoch 1/1
7200/7200 

In [68]:
best_accuracy

0.796

In [69]:
best_params

{'batch_size': 25, 'nb_epoch': 100, 'optimizer_arg': 'adam'}